In [2]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import requests
import folium
# !pip install pandas # 파이썬 데이터 처리 라이브러리 
import pandas as pd
from pandas.io.json import json_normalize # 파이썬 딕셔너리 타입의 데이터를 판다스 데이터 프레임으로 변환한다. 

전국 또는 특정 지역의 스타벅스 매작 위치 정보를 가져와서 지도위에 표시하기  
  
requests 모듈로 스타벅스 매장의 위치 정보를 가져와서 딕셔너리 타입으로 변환한다.    
json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.  
folium 모듈을 사용해서 지도를 표시하고 지도위에 스타벅스 매장 위치를 마커로 표시한다.  


In [4]:
# 서울 스타벅스 전 지점
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=A4HTV1LTE3'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, 
    'ins_lng': 126.9863309,
    'p_sido_cd': '01', 
    'p_gugun_cd': '',
    'in_biz_cd':'',
    'iend': 2000,
    'set_date': ''
})
starbucks = request.json()
print(type(starbucks))
#print(starbucks)

<class 'dict'>


In [5]:
# pandas.io.json 모듈의 json_normalize() 함수로 json 타입의 데이터가 변환된 딕셔너리를 판다스 데이터프레임으로 변환한다.
# json_nomalize(딕셔너리 ,'데이터프레임으로 변환할 데이터가 할당된 딕셔너리의 key')
starbucks_df = json_normalize(starbucks, 'list')
print(type(starbucks_df))
print(len(starbucks_df))
print(starbucks_df.columns)

<class 'pandas.core.frame.DataFrame'>
0
Index([], dtype='object')


In [9]:
starbucks_df

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,p90,p01,t05,t30,t36,t27,t29,t43,t48,z9999
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
597,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
598,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
599,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# 데이터프레임으로 변환하니 서울에 있는 스타벅스 지점은 601개고 각 지점당 131개의 컬럼으로 정보를 표시하고 있다 . 
# 작업에 필요한 컬럼 몇 가지를 선택해서 지도에 마커를 표시할 때 사용할 데이터가 저장된 데이터 프레임을 만든다.
# s_name => 지점이름 
# sido_cd => 시도코드
# sido_nm => 시도 이름
# gugun_cd => 구군 코드
# gugun_nm => 구군이름
# doro_address => 도로명주소
# lat => 위도
# lot => 경도

In [11]:
# 판다스 데이터프레임에서 특정 컬럼의 데이터를 얻어올 때 얻어올 컬럼이 1개라면 컬럼 이름만 적어도 되지만 얻어올 
# 컬럼이 2개 이상일 경우 얻어올 컬럼들을 반드시 리스트로 만들어서 얻어와야 한다.
starbucks_df_map = starbucks_df[
    [
        's_name',
        'sido_code',
        'sido_name',
        'gugun_code',
        'gugun_name',
        'doro_address',
        'lat',
        'lot'
    ]
]
starbucks_df_map

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
0,역삼아레나빌딩,01,서울,0101,강남구,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,01,서울,0101,강남구,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,01,서울,0101,강남구,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,국기원사거리,01,서울,0101,강남구,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩R,01,서울,0101,강남구,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...,...,...,...,...
596,사가정역,01,서울,0125,중랑구,서울특별시 중랑구 면목로 310,37.579594,127.087966
597,상봉역,01,서울,0125,중랑구,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
598,묵동,01,서울,0125,중랑구,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
599,양원역,01,서울,0125,중랑구,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [12]:
# dtype 속성으로 데이터 프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다 .
# 판다스는 문자열 데이터 타입을 object라 부른다.
starbucks_df_map.dtypes

s_name          object
sido_code       object
sido_name       object
gugun_code      object
gugun_name      object
doro_address    object
lat             object
lot             object
dtype: object

In [13]:
# info() 메소드를 사용해도 데이터프레임을 구성하는 컬럼들의 자료형을 확인할 수 있다 .
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   s_name        601 non-null    object
 1   sido_code     601 non-null    object
 2   sido_name     601 non-null    object
 3   gugun_code    601 non-null    object
 4   gugun_name    601 non-null    object
 5   doro_address  601 non-null    object
 6   lat           601 non-null    object
 7   lot           601 non-null    object
dtypes: object(8)
memory usage: 37.7+ KB


In [14]:
# astype() 메소드로 위도, 경도 데이터 타입을 object에서 float로 변환한다.
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucks_df_map.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 601 entries, 0 to 600
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   s_name        601 non-null    object 
 1   sido_code     601 non-null    object 
 2   sido_name     601 non-null    object 
 3   gugun_code    601 non-null    object 
 4   gugun_name    601 non-null    object 
 5   doro_address  601 non-null    object 
 6   lat           601 non-null    float64
 7   lot           601 non-null    float64
dtypes: float64(2), object(6)
memory usage: 37.7+ KB


In [15]:
# 종각점
starbucks_df_map[starbucks_df_map['s_name'] == '종각']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
443,종각,01,서울,0118,종로구,서울특별시 종로구 종로 64 (종로2가),37.569918,126.984528


In [16]:
starbucks_df_map[starbucks_df_map['s_name'] == '종로관철']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
445,종로관철,01,서울,0118,종로구,"서울특별시 종로구 종로12길 21, 2층 (관철동)",37.569058,126.986013


In [17]:
starbucks_df_map[starbucks_df_map['s_name'] == '종로구청']

,s_name,sido_code,sido_name,gugun_code,gugun_name,doro_address,lat,lot
430,종로구청,01,서울,0118,종로구,서울특별시 종로구 삼봉로 71 (수송동),37.572662,126.981138


In [18]:
starbucksMap = folium.Map(location=[37.56961146, 126.9840133], zoom_start=17)
popup = folium.Popup('스타벅스 종각점', max_width=300)
folium.Marker(location=[37.569918, 126.984528], popup=popup).add_to(starbucksMap)
popup = folium.Popup('스타벅스 종로관철점', max_width=300)
folium.Marker(location=[37.569058, 126.986013], popup=popup).add_to(starbucksMap)
popup = folium.Popup('스타벅스 종로구청점', max_width=300)
folium.Marker(location=[37.572662, 126.981138], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap1.html')
starbucksMap

In [19]:
# 종로구 스타벅스 전 
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=2EU3FUFOF1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '', # 시도코드
    'p_gugun_cd': '0118', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000,  # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[
    [
        's_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot'
    ]
]
starbucks_df_map
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=15)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} - {}'.format(index +1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker([data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
starbucksMap

In [20]:
# 종로구 스타벅스 전 
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=2EU3FUFOF1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '16', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000,  # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[
    [
        's_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot'
    ]
]
starbucks_df_map
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=11)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} - {}'.format(index +1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker([data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.html')
starbucksMap

In [21]:
# 종로구 스타벅스 전 
targetSite = 'https://www.starbucks.co.kr/store/getStore.do?r=2EU3FUFOF1'
request = requests.post(targetSite, data={
    'ins_lat': 37.563398, # 위도
    'ins_lng': 126.9863309, # 경도
    'p_sido_cd': '08', # 시도코드
    'p_gugun_cd': '', # 구군코드
    'in_biz_cd': '', # ?
    'iend': 2000,  # 서버가 응답하는 최대 매장의 개수
    'set_date': '' # ?
})
starbucks = request.json()
starbucks_df = json_normalize(starbucks, 'list')
starbucks_df_map = starbucks_df[
    [
        's_name', 'sido_code', 'sido_name', 'gugun_code', 'gugun_name', 'doro_address', 'lat', 'lot'
    ]
]
starbucks_df_map
starbucks_df_map['lat'] = starbucks_df_map['lat'].astype(float)
starbucks_df_map['lot'] = starbucks_df_map['lot'].astype(float)
starbucksMap = folium.Map(location=[starbucks_df_map['lat'].mean(), starbucks_df_map['lot'].mean()], zoom_start=15)
for index, data in starbucks_df_map.iterrows():
    popup = folium.Popup('{}. {} - {}'.format(index +1, data['s_name'], data['doro_address']), max_width=300)
    folium.Marker([data['lat'], data['lot']], popup=popup).add_to(starbucksMap)
starbucksMap.save('./starMap3.ntml')
starbucksMap